# For 1 percent data with 1000 points we are taking a dnn driven trajectory


In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn 
from keras.layers import Dense
from keras.models import Sequential
import os
import matplotlib.pyplot as plt

In [54]:
df


,h,v,s,omega,gamma,m,theta,Thrust,beta,time
0,4974.800000,323.3700,2.430700e-16,3.420000e-01,-1.3090,26230,-1.3090,479450.0,0.049258,0.000000
1,4967.900000,322.8800,1.851600e+00,3.404900e-01,-1.3092,26226,-1.3014,477910.0,0.048187,0.022150
2,4961.000000,322.4000,3.698800e+00,3.390200e-01,-1.3095,26222,-1.2939,476430.0,0.047199,0.044300
3,4954.100000,321.9200,5.541400e+00,3.375800e-01,-1.3097,26219,-1.2864,475020.0,0.046291,0.066451
4,4947.300000,321.4400,7.379600e+00,3.361700e-01,-1.3100,26215,-1.2790,473670.0,0.045462,0.088601
...,...,...,...,...,...,...,...,...,...,...
2189182,0.025833,5.0854,8.542800e+02,1.416700e-07,-1.5708,22080,-1.5708,585350.0,-0.000024,27.569000
2189183,0.019347,5.0640,8.542800e+02,1.881000e-07,-1.5708,22080,-1.5708,585290.0,-0.000018,27.571000
2189184,0.012889,5.0427,8.542800e+02,2.210900e-07,-1.5708,22080,-1.5708,585240.0,-0.000012,27.572000
2189185,0.006459,5.0213,8.542800e+02,2.407900e-07,-1.5708,22080,-1.5708,585190.0,-0.000006,27.573000


In [2]:
df = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/1000 points per trajectory/May10rd1percent_RLV_data 1000 points.csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
input = output =df.values
X = input[:,0:7]
y = output[:,7:9]
# Individual Data
from sklearn import preprocessing
X_norm = preprocessing.minmax_scale(X)
y_norm = preprocessing.minmax_scale(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.1, random_state=42)
X_val = X_train[-10000:]
y_val = y_train[-10000:]
X_train = X_train[:-10000]
y_train = y_train[:-10000]

In [3]:
def create_model2(n):
    model = Sequential()
    model.add(Dense(n,input_shape=(7,),kernel_initializer='uniform'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu')) # since tanh has more nonlinearity we add it here, it also gives -ve values so , some layers which are not necessary will lead to 0 in next layer
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(2,kernel_initializer='uniform',activation = 'sigmoid'))
    return model

In [4]:
i = 7;
model = create_model2(2**i)
path_Of_model =f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/1new percent variation/1000 points 7 states and 2 op with val_set having 128 neurons and batchsize 256/cp.ckpt"
model.load_weights(path_Of_model)

2022-05-20 16:59:47.977977: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-05-20 16:59:47.978026: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-05-20 16:59:47.979224: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.mean_squared_error
model.compile(optimizer=opt,loss=loss,metrics='accuracy')
# loss,accuracy = model.evaluate(X_norm,y_norm)

# Values for DNN driven trajectory

In [6]:
import math

In [7]:
R0 = 6378*1000;
g0=9.81
Cd = 0.5
S_ref =10.75
Isp =300;
I_z = 3346393
l_com = 9.778
t_ref = 32
h0=5000;
v0=325;
s0=850;
## The inputs and outputs which varied have values in decimals 
## rest all have been rounded off to 5-6 digits
omega0= 0.342
theta0= round(-75*math.pi/180,4)
gamma0= round(-75*math.pi/180,4) #
m0= round(26229.667,0) # our values of mass are rounded of when taking data
r0 = h0+637800

max(df.h.values)

df.loc[df['s']==850]

## Here we are finding the trajectory which relates to the given input conditions

In [8]:
# First we find the index of the row where we have the given initial conditions, we validate if we had 
# taken the correct trajectory for that we need to check the ending values of s such that s = s0.
idx = df.loc[(df['h'] == h0) & (df['v'] == v0)]
idx = np.array(idx.index.astype(int))
def trajectory_finding(idx,df,s0):
    # we have 3003 points for getting full trajectory starting from index 0-3002
    for i in range(idx.shape[0]):
        df2= df[idx[0]+3003*i:idx[0]+3003*(i+1)]
        if (df2.s.values[3002]==s0):
            break
        else:
            continue
    return np.array([df2.index[0],df2.index[3002]])

In [51]:
idx_i,idx_f = trajectory_finding(idx,df,s0)

In [52]:
idx_i

1093092

In [10]:
## Normalizing the inputs
def norm_inputs(a,b):
    return (a-min(b))/(max(b)-min(b))

In [11]:
h = X[:,0];
v = X[:,1];
s = X[:,2];
omega = X[:,3]
gamma= X[:,4]
m = X[:,5]
theta = X[:,6]
T = y[:,0]
b = y[:,1]
r = 637800+h

In [12]:
h_norm = norm_inputs(h,h)
v_norm= norm_inputs(v,v)
s_norm= norm_inputs(s,s)
omega_norm= norm_inputs(omega,omega)
gamma_norm= norm_inputs(gamma,gamma)
m_norm= norm_inputs(m,m)
theta_norm= norm_inputs(theta,theta)
r_norm = norm_inputs(r,r)
T_norm = norm_inputs(T,T)
b_norm = norm_inputs(b,b)

In [13]:
h1 = norm_inputs(h0,h)
v1= norm_inputs(v0,v)
s1= norm_inputs(s0,s)
omega1= norm_inputs(omega0,omega)
gamma1= norm_inputs(gamma0,gamma)
m1= norm_inputs(m0,m)
theta1= norm_inputs(theta0,theta)
r1 = norm_inputs(r0,r)

In [14]:
math.pi/180

0.017453292519943295

In [15]:
Thrust


NameError: name 'Thrust' is not defined

In [17]:
# For this trajectory we are creating a vector of Neural Network inputs and obtain the Thrust, beta predicted values.
# given all the 7 states as input. and these controls will be sent to the DNN driven trajectory generation model
# ip_traj = df.all
# normalize the inputs and ouputs for observation
data_norm = np.array([h_norm,v_norm,s_norm,omega_norm,gamma_norm,m_norm,theta_norm,T_norm,b_norm]).T
df_norm = pd.DataFrame(data_norm,columns=['h','v','s','omega','gamma','m','theta','Thrust','beta'])

# Trajectory for the given initial condition:
data_traj = df_norm.values[idx_i:idx_f+1][:,0:9]
ip_traj = data_traj[:,0:7]
op_traj = data_traj[:,7:9]

# Intermediate trajectories averaged initial guess 

In [18]:
## Intermediate trajectory values:
ip_mid = [];
op_mid = [];
for i in range(len(ip_traj)-1):
     ip_mid = np.append(ip_mid,(ip_traj[i,:]+ip_traj[i+1,:])/2)
     op_mid = np.append(op_mid,(op_traj[i,:]+op_traj[i+1,:])/2)
        
ip_mid = ip_mid.reshape(len(ip_traj)-1,7)
op_mid = op_mid.reshape(len(ip_traj)-1,2)

In [19]:
"""
h_traj = np.where(h1==h_norm)
v_traj = np.where(v1==v_norm)
s_traj = np.where(s1==s_norm)
omega_traj = np.where(omega1==omega_norm)
gamma_traj = np.where(gamma1 == gamma_norm)
m_traj = np.where(m1==m_norm)
theta_traj=np.where(theta1==theta_norm)
# T_traj=np.where(T1==T_norm)
# b_traj=np.where(b1==b_norm)
"""

'\nh_traj = np.where(h1==h_norm)\nv_traj = np.where(v1==v_norm)\ns_traj = np.where(s1==s_norm)\nomega_traj = np.where(omega1==omega_norm)\ngamma_traj = np.where(gamma1 == gamma_norm)\nm_traj = np.where(m1==m_norm)\ntheta_traj=np.where(theta1==theta_norm)\n# T_traj=np.where(T1==T_norm)\n# b_traj=np.where(b1==b_norm)\n'

## Now we need to append the values for all state variables in DNN driven trajectory and then we need to antinormalize the outputs to validate with the actual trajectory.

In [20]:
def AntiNorm(Norm_value,actual):
  return (max(actual)-min(actual))*Norm_value+min(actual)

AntiNorm(T,T)
AntiNorm(b,b)


array([-0.17027757, -0.170684  , -0.17105892, ..., -0.18897457,
       -0.18897227, -0.18897   ])

## Defining variables which will be appended as the iterations progress in RK4 method:

In [21]:
# local variables within the DNN function
ω2 = [];
θ2 = [];
v2 = [];
γ2 = [];
r2 = [];
s2 = [];
h2 = [];

In [22]:
# global variables for appending
ω_dnn = [];
θ_dnn = [];
v_dnn = [];
γ_dnn = [];
r_dnn = [];
s_dnn = [];
h_dnn = [];

h_dnn=5
for _ in range(5):
    h_dnn = np.append(h_dnn,_)
    h_dnn
h_dnn

In [23]:
ip = np.array([h1,v1,s1,omega1,gamma1,m1,theta1]).reshape(1,7)
ip_int = ip_mid[0,:]
op_int = op_mid[0,:]

In [24]:
T1,b1 = np.squeeze(model.predict(ip))
T1_int,b1_int = np.squeeze(model.predict(ip_int.reshape(1,7)))
print(T1,b1, T1_int,b1_int)

1.7029949e-08 1.2901937e-09 0.24050793 0.62022


In [33]:
## Normalizing the inputs
def norm(a,b):
    return (a - min(b))/(max(b) - min(b))

## Substituting the actual values and validating if we get the same DNN

In [25]:
T1,b1 = op_traj[0,:] # 1st row of outputs for the given inputs
T1_int,b1_int = op_mid[0,:] # 1st row of intermediate values
print(T1,b1, T1_int,b1_int)

0.29138459162398 0.6510435332560345 0.2897757986048547 0.6501396648044693


## Generating the trajectory from outputs using RK4

In [34]:
def diff_eqn2(inputs,model): # sends norm inputs
        """ Debugg ω, γ"""
        r1 = inputs[0]
        s1 = inputs[1]
        v1 = inputs[2]
        gamma1 = inputs[3]
        m1 = inputs[4]
        theta1= inputs[5]
        omega1 = inputs[6]
        
        ip = np.array([r1,v1,s1,omega1,gamma1,m1,theta1]).reshape(1,7) 
        data = model.predict(ip)
        T1,beta = np.squeeze(data)
        
        r1 = AntiNorm(r1,r)
        s1 = AntiNorm(s1,s)
        v1 = AntiNorm(v1,v)
        gamma2 = AntiNorm(gamma1,gamma)
        m1 = AntiNorm(m1,m)
        theta2 = AntiNorm(theta1,theta)
        omega2 = AntiNorm(omega1,omega)
        
        T1 = AntiNorm(T1,T)
        beta = AntiNorm(beta,b)
        
        ## normalizing according to the equations :
        r1 = (r1)/R0; # here r1 is already R0+h
        s1 = s1/s0;
        v1 = v1/v0;
        m1 = m1/m0;
        T1 = T1/(m0*g0)

        y1 = np.array([r1,s1,v1,gamma2,m1,theta2,omega2]) # anti norms
#         print("old n states",(r1,s1,v1,gamma2,m1,theta2,omega2))

        dy = np.zeros(7)
        Dr= 0.5*1.225*((y1[2]*v0)**2)*Cd*S_ref/(m0*g0);
        dy[0] = y1[2] * np.sin(y1[3])*(v0*t_ref/R0); 
        dy[1] = y1[2] * np.cos(y1[3])*(v0*t_ref/s0);    
        dy[2] = - (((T1* np.cos(beta-y1[3] + y1[5])+Dr)/y1[4]) + (np.sin(y1[3])/(y1[0]**2)))*(g0*t_ref/v0);
        dy[3] = - ((T1 * np.sin(beta-y1[3] + y1[5])/(y1[4]*y1[2])) + (np.cos(y1[3])/((y1[0]**2)*y1[2])))*(g0*t_ref/v0);
        dy[4] = -T1/Isp*t_ref;
        dy[5] = y1[6]*t_ref; 
        dy[6] = -T1 * np.sin(beta)*l_com*(m0*g0*t_ref)/I_z
        
        return dy,y1,T,norm(beta,b) 

In [35]:
def Euler(input2,model,n):
    global y1, y2, k1, y3
    h_step = 1/n
    inputs= np.array([r1,s1,v1,gamma1,m1,theta1,omega1])
    op_dnn = np.append([],inputs) 
    op_an = []
    for _ in range(n):
        k1,y1,T1,b1 = diff_eqn2(inputs,model)
        op_an = np.append(op_an,y1)
        y2 = y1 + h_step *(k1) # euler 
        """change y2 to normalized and append to op
           y1 = np.array([r1,s1,v1,gamma2,m1,theta2,omega2]) """
        y2[3] = norm(y2[3],gamma)
        y2[5] = norm(y2[-2],theta)
        y2[6] = norm(y2[-1],omega)
        op_dnn = np.append(op_dnn,y2) 
        # Normalizing them back to the way our neural network will be able to predict
        y2[0] = norm((y2[0])*R0,r) 
        y2[1] = norm(y2[1]*s0,s)
        y2[2] = norm(y2[2] *v0,v)
        y2[4] = norm(y2[4] *m0,m)
        print("AFTER",y2)
        inputs = y2 
    return op_an,op_dnn

In [36]:
def Rk4(input2,model,n):
    global y1, y2, k1,k2,k3,k4, y3
    h_step = 1/n
    inputs= np.array([r1,s1,v1,gamma1,m1,theta1,omega1])
    op_dnn = np.append([],inputs) 
    op_an = []   
    for _ in range(n):
        k1,y1,T1,b1 = diff_eqn2(inputs,model)
        k2,_,_,_ = diff_eqn2(inputs+0.5*h_step*k1,model)
        k3,_,_,_ = diff_eqn2(inputs+0.5*h_step*k2,model)
        k4,_,_,_ = diff_eqn2(inputs+h_step*k3,model)
    
        y2 = y1+ h_step *(k1+2*(k2+k3)+k4)/6 
        
        """change y2 to normalized and append to op
           y1 = np.array([r1,s1,v1,gamma2,m1,theta2,omega2]) """
        
        # Normalizing them back to the way our neural network will be able to predict
        y2[0] = norm((y2[0])*R0,r) 
        y2[1] = norm(y2[1]*s0,s)
        y2[2] = norm(y2[2] *v0,v)
        y2[3] = norm(y2[3],gamma)
        y2[4] = norm(y2[4] *m0,m)
        y2[5] = norm(y2[5],theta)
        y2[6] = norm(y2[6],omega)
        ## 
#         print("AFTER",y2)
        inputs = y2 
        op_dnn = np.append(op_dnn,y2) # appending trajectory states obtained by ddnn
    
    return op_an,op_dnn

In [48]:
f1,f2= Euler(np.array([r1,s1,v1,gamma1,m1,theta1,omega1]),model,10)

AFTER [ 0.79508028  1.29492951 10.00742417 -3.4957906   0.91424328  1.16712799
  1.79907965]
AFTER [ 1.95862156 -8.53847102 -8.51584027 -2.16835948  0.82848657  2.72850766
  0.95109697]
AFTER [  2.30009641   1.38748875 -16.40773695  -3.99551148   0.74272985
   3.26025678   1.79230705]
AFTER [ -0.33768276  13.46350049 -60.17385807  -4.5546677    0.5665202
   4.81341308   3.5207937 ]
AFTER [-1.19939828e+01  3.66771865e+01 -5.76171355e+02 -4.65035004e+00
  3.90310546e-01  8.46531823e+00  5.24928034e+00]
AFTER [-1.25648420e+02  2.21221462e+02 -4.86913609e+04 -4.65959015e+00
  2.14100892e-01  1.42159722e+01  6.97776699e+00]
AFTER [-9.74545611e+03  1.55059373e+04 -3.56231674e+08 -5.53278207e+00
  3.78912382e-02  2.20653750e+01  8.70625364e+00]
AFTER [-7.05267102e+07 -1.08845628e+08 -1.97814407e+16 -5.53278206e+00
 -4.78654770e-02  3.20135267e+01  9.54746372e+00]
AFTER [-3.91578651e+15 -6.04502705e+15 -6.21419391e+31 -5.53278206e+00
 -1.33622192e-01  4.29830855e+01  1.03886738e+01]
AFTER [-1.

In [49]:
f2 = f2.reshape(11,7)

In [50]:
f2[-1,:]

array([-9.69203962e+27, -1.93088157e+31, -6.18502283e+62, -5.53278206e+00,
        7.82901277e-01,  5.49740516e+01,  1.12298839e+01])